*Note: this notebook was generated by dsflow*

# create table camera_agg

## \[1- init and config\] load tables and set output path

In [1]:
import os
import json
import datetime as dt
from pandasql import PandaSQL
from custom_libraries.helpers import show_tables, load_table

In [2]:
# default task parameters
default_task_specs = """
    {"sink_path": "/data/tables/camera_agg/ds=2018-02-06",
     "ds": "2018-02-06"}
    """

task_specs = json.loads(os.environ.get('TASK_SPECS', default_task_specs))

## \[2- process data\] process data based on SQL

In [3]:
# show tables (parquet files located in datastore/tables/*)
show_tables()

camera_agg
car_speed_limits
car_speed_limits_agg
dim_users
meteo_agg
meteoparis
users_agg


In [4]:
dim_cameras = load_table(type="json", table_name="dim_cameras")

In [5]:
dim_cameras.head()

,camera_uuid,creation_ts,manufacturing_id,model,modification_ts,owner_user_uuid
0,992eeced-2520-4a3e-b74e-137439e97f71,2017-08-27T05:36:06.377Z,POQOGDF1,F2,2018-01-23T14:54:43.372Z,90b14334-ff95-4e4a-9b24-5da0027f2311
1,8570a8c8-36da-42ab-8b0b-4201a18599d4,2017-04-02T10:04:58.062Z,PFNECXT1,F2,2018-01-23T20:56:04.892Z,57149a90-17a7-470c-96c8-d4faf63ab8db
2,685a0408-0b5e-42c5-b0ac-5c635d6bcd12,2018-01-11T04:13:23.717Z,HMTOSSG1,F2,2018-01-23T15:39:24.712Z,5754d704-1ffd-4278-97bc-f751ea46c4c5
3,9f864bd4-d063-4ae9-9532-bcc14f111c0b,2017-11-11T20:31:48.340Z,BZQETDU1840,F2,2018-01-23T09:15:21.250Z,d543832c-5147-4989-84bb-8fe0592a3fe5
4,17e4a7bd-8231-4b6a-a571-d3ce384472e8,2017-10-12T08:45:33.164Z,NUWAVEP1KUF,M1,2018-01-23T14:11:06.876Z,097b272f-79b6-4900-9bc9-cb9c29ae1100


In [6]:
psql = PandaSQL()

In [7]:
camera_agg_df = psql("""

  SELECT model, count(*) as model_count
  FROM dim_cameras
  group by 1
  LIMIT 100

""")

camera_agg_df

,model,model_count
0,E3,610
1,F2,517
2,M1,535
3,V4,538


In [8]:
camera_agg_df.describe()

,model_count
count,4.00000
mean,550.00000
std,41.06093
min,517.00000
25%,530.50000
50%,536.50000
75%,556.00000
max,610.00000


## \[3- write on disk\] write the output as parquet

In [9]:
base_dir = "/data/tables/camera_agg/ds=2018-02-06"

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

camera_agg_df.to_parquet(task_specs["sink_path"] + "/data.parquet")